# Analysis of early 2020 Democratic campaign co-donors

This notebook analyzes contribution data from Democratic presidential campaigns' FEC filings for the first quarter of 2019.

*Added by Steve*

See article [BuzzFeed News article examining donors](https://www.buzzfeednews.com/article/tariniparti/democratic-donors-2020-candidates). 

This article analyzed how a money is distributed

## Load candidate, committee, and filing data

In [1]:
import pandas as pd
import fecfile

*Added by Steve*

See more about `fecfile` at https://esonderegger.github.io/fecfile/

### Candidates

In [2]:
candidates = pd.read_csv("../inputs/candidates.csv")

candidates

,Candidate Name,Candidate Sex,Committee Name,committee_id
0,Amy Klobuchar,F,Amy for America,C00696419
1,Andrew Yang,M,Friends of Andrew Yang,C00659938
2,Bernie Sanders,M,Bernie 2020,C00696948
3,Beto O'Rourke,M,Beto for America,C00699090
4,Cory Booker,M,Cory 2020,C00695510
5,Elizabeth Warren,F,Warren for President,C00693234
6,Jay Inslee,M,Inslee for America,C00698050
7,John Delaney,M,Friends of John Delaney,C00508416
8,John Hickenlooper,M,Hickenlooper 2020,C00698258
9,Julián Castro,M,Julián for the Future,C00693044


### Filing metadata

Here, we load basic metadata about each filing, and also calculate what proportion of money raised from individual contributions has been itemized in each candidate/committee's filings:

In [3]:
filings = pd.read_csv(
    "../inputs/filings.csv",
    low_memory = False
)

filings

,Candidate Name,committee_id,filing_id,report_title,date_coverage_from,date_coverage_to,amended
0,Amy Klobuchar,C00696419,1326529,APR QUARTERLY,2019-02-07,2019-03-31,False
1,Andrew Yang,C00659938,1326379,APR QUARTERLY,2019-01-01,2019-03-31,False
2,Bernie Sanders,C00696948,1326070,APR QUARTERLY,2019-01-01,2019-03-31,False
3,Beto O'Rourke,C00699090,1326481,APR QUARTERLY,2019-03-14,2019-03-31,False
4,Cory Booker,C00695510,1326465,APR QUARTERLY,2019-01-01,2019-03-31,False
5,Elizabeth Warren,C00693234,1326299,APR QUARTERLY,2019-01-01,2019-03-31,False
6,Jay Inslee,C00698050,1326136,APR QUARTERLY,2019-02-14,2019-03-31,False
7,John Delaney,C00508416,1324749,APR QUARTERLY,2019-01-01,2019-03-31,False
8,John Hickenlooper,C00698258,1326014,APR QUARTERLY,2019-01-01,2019-03-31,False
9,Julián Castro,C00693044,1326324,APR QUARTERLY,2019-01-01,2019-03-31,False


In [6]:
def get_additional_metadata(filing_id):
    filing = fecfile.from_file(f"../inputs/filings/{int(filing_id)}.fec")
    data = dict((c, filing["filing"][c]) for c in [
        "col_a_individuals_itemized",
        "col_a_individuals_unitemized",
        "col_a_individual_contribution_total",
    ])
    data["filing_id"] = int(filing_id)
    return data

In [7]:
additional_filing_metadata = pd.DataFrame([
    get_additional_metadata(filing_id)
    for filing_id in filings["filing_id"]
])

additional_filing_metadata.head()

,col_a_individual_contribution_total,col_a_individuals_itemized,col_a_individuals_unitemized,filing_id
0,5232375.87,3421762.07,1810613.80,1326529
1,1776875.12,342170.20,1434704.92,1326379
2,18186300.21,2904271.23,15282028.98,1326070
3,9369861.40,3827220.52,5542640.88,1326481
4,5044390.15,4238894.87,805495.28,1326465


In [8]:
# Added by Steve

additional_filing_metadata['is_equal'] = (additional_filing_metadata.col_a_individuals_itemized + \
    additional_filing_metadata.col_a_individuals_unitemized) \
    == additional_filing_metadata.col_a_individual_contribution_total

additional_filing_metadata

,col_a_individual_contribution_total,col_a_individuals_itemized,col_a_individuals_unitemized,filing_id,is_equal
0,5232375.87,3421762.07,1810613.80,1326529,True
1,1776875.12,342170.20,1434704.92,1326379,False
2,18186300.21,2904271.23,15282028.98,1326070,True
3,9369861.40,3827220.52,5542640.88,1326481,True
4,5044390.15,4238894.87,805495.28,1326465,True
5,6016435.38,1786711.53,4229723.85,1326299,True
6,2255455.41,1488634.36,766821.05,1326136,True
7,404301.17,331244.84,73056.33,1324749,False
8,2014099.37,1813358.33,200741.04,1326014,True
9,1092941.02,719775.12,373165.90,1326324,True


In [8]:
(
    filings
    [[
        "filing_id",
        "Candidate Name",
    ]]
    .merge(
        additional_filing_metadata
        [[
            "filing_id",
            "col_a_individuals_itemized",
            "col_a_individuals_unitemized",
            "col_a_individual_contribution_total",
        ]]
        .assign(
            prop_itemized = lambda df: (
                df["col_a_individuals_itemized"] / 
                df["col_a_individual_contribution_total"]
            ).round(3)
        ),
        how = "right",
        on = "filing_id"
    )
    .sort_values("col_a_individuals_itemized", ascending = False)
)

,filing_id,Candidate Name,col_a_individuals_itemized,col_a_individuals_unitemized,col_a_individual_contribution_total,prop_itemized
10,1326016,Kamala Harris,7603293.36,4420828.19,12024121.55,0.632
4,1326465,Cory Booker,4238894.87,805495.28,5044390.15,0.840
3,1326481,Beto O'Rourke,3827220.52,5542640.88,9369861.40,0.408
0,1326529,Amy Klobuchar,3421762.07,1810613.80,5232375.87,0.654
2,1326070,Bernie Sanders,2904271.23,15282028.98,18186300.21,0.160
12,1324922,Pete Buttigieg,2549602.40,4536552.22,7086154.62,0.360
11,1326061,Kirsten Gillibrand,2497960.90,499923.20,2997884.10,0.833
8,1326014,John Hickenlooper,1813358.33,200741.04,2014099.37,0.900
5,1326299,Elizabeth Warren,1786711.53,4229723.85,6016435.38,0.297
6,1326136,Jay Inslee,1488634.36,766821.05,2255455.41,0.660


### Contributors

Here, we extract contributions to the committees from individuals, and assign `donor_id`s: a combination of the contributor's listed first name, last name, and five-digit ZIP code.

In [9]:
def make_donor_ids(df):
    return (
        df
        .assign(
            donor_id = lambda df: (
                df
                .assign(
                    zip5 = lambda df: (
                        df["contributor_zip_code"]
                        .fillna("-----")
                        .str.slice(0, 5)
                    )
                )
                [[
                    "contributor_first_name",
                    "contributor_last_name",
                    "zip5",
                ]]
                .apply(lambda x: (
                    x
                    .fillna("")
                    .astype(str)
                    # Remove periods, commas, extra whitespace
                    .str.replace(r"[\.,\s]+", " ")
                    .str.strip()
                    # Convert everything to upper-case
                    .str.upper()
                ))
                .apply("|".join, axis = 1)
            )
        )
    )

In [10]:
def extract_indiv_contributions(filing_id):
    filing = fecfile.from_file(f"../inputs/filings/{int(filing_id)}.fec")
    df = pd.DataFrame(filing["itemizations"]["Schedule A"])
    return (
        df
        # Extract only individual contributions
        .loc[lambda df: df["entity_type"] == "IND"]
        # Remove memo lines
        .loc[lambda df: df["memo_code"] == ""]
        .pipe(make_donor_ids)
        .assign(
            filing_id = int(filing_id)
        )
        [[
            "filer_committee_id_number",
            "filing_id",
            "transaction_id",
            "contribution_date",
            "contribution_amount",
            "contribution_aggregate",
            "donor_id",
            "contributor_first_name",
            "contributor_last_name",
            "contributor_zip_code",
        ]]
    )

In [9]:
# Added by Steve
pd.concat([
    extract_indiv_contributions(filing_id)
    for filing_id in filings["filing_id"]
])

,filer_committee_id_number,filing_id,transaction_id,contribution_date,contribution_amount,contribution_aggregate,donor_id,contributor_first_name,contributor_last_name,contributor_zip_code
0,C00696419,1326529,561500,2019-02-20 00:00:00-05:00,2800.00,5600.00,JUSTINE|HASELOW|55436,Justine,Haselow,554361001
1,C00696419,1326529,564400,2019-02-19 00:00:00-05:00,2800.00,2800.00,MARTIN|CARLSON|55401,Martin,Carlson,554012159
3,C00696419,1326529,564500,2019-02-18 00:00:00-05:00,250.00,250.00,WILLIAM|BELL|90068,William,Bell,900682646
5,C00696419,1326529,565000,2019-02-19 00:00:00-05:00,100.00,250.00,REBECCA|BEATTIE|80007,Rebecca,Beattie,800076971
7,C00696419,1326529,566000,2019-02-19 00:00:00-05:00,25.00,325.00,CAROLINE|TOWER|94115,Caroline,Tower,941152418
9,C00696419,1326529,588000,2019-03-19 00:00:00-04:00,15.00,215.00,ROLLYN|STRAND|56225,Rollyn,Strand,562250418
11,C00696419,1326529,552000,2019-02-10 00:00:00-05:00,100.00,250.00,STEVEN|WAYLAND|60614,Steven,Wayland,606141116
13,C00696419,1326529,552900,2019-02-10 00:00:00-05:00,100.00,262.00,JAMES|MORRIS|30338,James,Morris,303384548
15,C00696419,1326529,554100,2019-02-10 00:00:00-05:00,500.00,500.00,WARREN|TRIPP|53711,Warren,Tripp,537115288
17,C00696419,1326529,555300,2019-02-10 00:00:00-05:00,250.00,250.00,ALEC|BRINDLE|98199,Alec,Brindle,981993802


In [12]:
all_indiv_contribs = (
    
    # Steve: Union "tables" of individual filings into one master table
    pd.concat([
        extract_indiv_contributions(filing_id)
        for filing_id in filings["filing_id"]
    ])
    
    # Steve: Add filing metadata to individual records
    .merge(
        (
            filings
            [[
                "filing_id",
                "committee_id",
            ]]
        ),
        how = "left",
        on = "filing_id",
    )
    
    # Steve: An more semantic metadata to individual records
    .merge(
        (
            candidates
            [[
                "committee_id",
                "Candidate Name",
            ]]
        ),
        how = "left",
        on = "committee_id",
    )
)

len(all_indiv_contribs)

92830

In [13]:
all_indiv_contribs.head()

,filer_committee_id_number,filing_id,transaction_id,contribution_date,contribution_amount,contribution_aggregate,donor_id,contributor_first_name,contributor_last_name,contributor_zip_code,committee_id,Candidate Name
0,C00696419,1326529,561500,2019-02-20 00:00:00-05:00,2800.0,5600.0,JUSTINE|HASELOW|55436,Justine,Haselow,554361001,C00696419,Amy Klobuchar
1,C00696419,1326529,564400,2019-02-19 00:00:00-05:00,2800.0,2800.0,MARTIN|CARLSON|55401,Martin,Carlson,554012159,C00696419,Amy Klobuchar
2,C00696419,1326529,564500,2019-02-18 00:00:00-05:00,250.0,250.0,WILLIAM|BELL|90068,William,Bell,900682646,C00696419,Amy Klobuchar
3,C00696419,1326529,565000,2019-02-19 00:00:00-05:00,100.0,250.0,REBECCA|BEATTIE|80007,Rebecca,Beattie,800076971,C00696419,Amy Klobuchar
4,C00696419,1326529,566000,2019-02-19 00:00:00-05:00,25.0,325.0,CAROLINE|TOWER|94115,Caroline,Tower,941152418,C00696419,Amy Klobuchar


## Aggregate contributions to donor-campaign level

The raw FEC data includes one row for each contribution, so contributors can show up multiple times for a given campaign. Here, we aggregate the data so that it has just one row per contributor-campaign combination:

In [14]:
# There appear to be some donors who've been refunded to $200 or less
(
    all_indiv_contribs
    .loc[lambda df: df["contribution_aggregate"] <= 200]
    ["donor_id"]
    .nunique()
)

519

In [15]:
contributor_totals = (
    all_indiv_contribs
    # Line below removes donors who appear to have been refunded
    # to $200 aggregate or less
    .loc[lambda df: df["contribution_aggregate"] > 200]
    .groupby([
        "donor_id",
        "Candidate Name"
    ])
    ["contribution_amount"]
    .sum()
    .reset_index()
)

contributor_totals.head()

,donor_id,Candidate Name,contribution_amount
0,0-DEREK|EILER|30306,Pete Buttigieg,250.0
1,A - DANA SMITH|SMITH|80534,Bernie Sanders,250.0
2,A ALEX|LARI|10128,Kirsten Gillibrand,2700.0
3,A C|HUDGINS|10025,Cory Booker,250.0
4,A J|AGUILA|07631,Andrew Yang,250.0


## Distinct donor counts, by candidate

In [16]:
distinct_donor_counts = (
    contributor_totals
    ["Candidate Name"]
    .value_counts()
    .to_frame("Distinct Donor IDs")
)

distinct_donor_counts

,Distinct Donor IDs
Bernie Sanders,9321
Kamala Harris,7489
Beto O'Rourke,4879
Pete Buttigieg,4045
Elizabeth Warren,3177
Cory Booker,3063
Amy Klobuchar,2867
Kirsten Gillibrand,1723
Jay Inslee,1214
John Hickenlooper,1090


## Find donors who gave to any two candidates, and any three candidates

In [23]:
# Added by Steve, what did the dataset look like before the self join?

contributor_totals[['donor_id', 'Candidate Name']].head()

,donor_id,Candidate Name
0,0-DEREK|EILER|30306,Pete Buttigieg
1,A - DANA SMITH|SMITH|80534,Bernie Sanders
2,A ALEX|LARI|10128,Kirsten Gillibrand
3,A C|HUDGINS|10025,Cory Booker
4,A J|AGUILA|07631,Andrew Yang


In [27]:
# Added by Steve, what did the dataset look like after the self join?
contributor_totals[['donor_id', 'Candidate Name']].pipe(lambda df: (
    df.merge(df, how='left', on='donor_id', suffixes = ['_x', '_y'])
)).loc[lambda df: df["Candidate Name_x"] != df["Candidate Name_y"]]

,donor_id,Candidate Name_x,Candidate Name_y
35,AARON|KATZEL|10013,Jay Inslee,Kamala Harris
36,AARON|KATZEL|10013,Kamala Harris,Jay Inslee
125,ABRAHAM|REICH|19102,Amy Klobuchar,Jay Inslee
126,ABRAHAM|REICH|19102,Jay Inslee,Amy Klobuchar
195,ADAM|INGERSOLL|52246,Kamala Harris,Pete Buttigieg
196,ADAM|INGERSOLL|52246,Pete Buttigieg,Kamala Harris
210,ADAM|LECLAIR|20003,Julián Castro,Kamala Harris
211,ADAM|LECLAIR|20003,Kamala Harris,Julián Castro
214,ADAM|LEVIN|10036,Cory Booker,Elizabeth Warren
215,ADAM|LEVIN|10036,Elizabeth Warren,Cory Booker


In [17]:
candidate_pairs = (
    contributor_totals
    .rename(columns = {
        "Candidate Name": "candidate"
    })
    [[
        "donor_id",
        "candidate"
    ]]
    .pipe(lambda df: (
        df
        .merge(
            df,
            how = "left",
            on = "donor_id",
            suffixes = [ "_x", "_y" ],
        )
    ))
    # This filter prevents us from double-counting candidate-combinations
    .loc[lambda df: df["candidate_x"] < df["candidate_y"]]
    .sort_values([
        "candidate_x",
        "candidate_y",
        "donor_id"
    ])
)

candidate_pairs.head(10)

,donor_id,candidate_x,candidate_y
8287,COLLIER|PERRY|76567,Amy Klobuchar,Andrew Yang
18921,JEAN|YNGVE|46304,Amy Klobuchar,Bernie Sanders
23339,KAREN|ALLIN|30305,Amy Klobuchar,Bernie Sanders
24146,KATHY|GIBBONS|20008,Amy Klobuchar,Bernie Sanders
29269,MARK|MOLLOY|50214,Amy Klobuchar,Bernie Sanders
29344,MARK|ROTHACHER|84117,Amy Klobuchar,Bernie Sanders
29456,MARK|WIZNITZER|22205,Amy Klobuchar,Bernie Sanders
34001,PARKE|CAPSHAW|22902,Amy Klobuchar,Bernie Sanders
963,ALICE|JARCHO|10065,Amy Klobuchar,Beto O'Rourke
1480,AMY|LOFGREN|85250,Amy Klobuchar,Beto O'Rourke


In [34]:
candidate_triplets = (
    contributor_totals
    .rename(columns = {
        "Candidate Name": "candidate"
    })
    [[
        "donor_id",
        "candidate"
    ]]
    .pipe(lambda df: (
        df
        .merge(
            df,
            how = "left",
            on = "donor_id",
            suffixes = [ "_x", "_y" ],
        )
        .merge(
            df.rename(columns = { "candidate": "candidate_z" }),
            how = "left",
            on = "donor_id",
        )
    ))
    # This filter prevents us from double-counting candidate-combinations
    .loc[lambda df: df["candidate_x"] < df["candidate_y"]]
    .loc[lambda df: df["candidate_y"] < df["candidate_z"]]
    .sort_values([
        "candidate_x",
        "candidate_y",
        "candidate_z",
        "donor_id"
    ])
)

candidate_triplets.head(10)

,donor_id,candidate_x,candidate_y,candidate_z
10547,COLLIER|PERRY|76567,Amy Klobuchar,Andrew Yang,Elizabeth Warren
43322,PARKE|CAPSHAW|22902,Amy Klobuchar,Bernie Sanders,John Hickenlooper
37377,MARK|MOLLOY|50214,Amy Klobuchar,Bernie Sanders,Kamala Harris
43323,PARKE|CAPSHAW|22902,Amy Klobuchar,Bernie Sanders,Kirsten Gillibrand
29581,KAREN|ALLIN|30305,Amy Klobuchar,Bernie Sanders,Pete Buttigieg
31040,KEENAN|KELSEY|94939,Amy Klobuchar,Beto O'Rourke,Cory Booker
5635,BILL|SIMS|75209,Amy Klobuchar,Beto O'Rourke,Elizabeth Warren
39698,MICHAEL|AUERBACH|10013,Amy Klobuchar,Beto O'Rourke,Elizabeth Warren
2275,ANDREW|FREDMAN|33156,Amy Klobuchar,Beto O'Rourke,John Hickenlooper
2445,ANDREW|MELLETT|90004,Amy Klobuchar,Beto O'Rourke,John Hickenlooper


## Identify the most common two-candidate combinations

Here, we count how many times donors has given to both Candidate X and Candidate Y, irrespective of any other contributions they might have made:

In [31]:
# Added by Steve

candidate_pairs.groupby(['candidate_x', 'candidate_y']).count()

donor_id
candidate_x        candidate_y                 
Amy Klobuchar      Andrew Yang                1
                   Bernie Sanders             7
                   Beto O'Rourke             65
                   Cory Booker               43
                   Elizabeth Warren          52
                   Jay Inslee                24
                   John Delaney               5
                   John Hickenlooper         16
                   Julián Castro              7
                   Kamala Harris            141
                   Kirsten Gillibrand        72
                   Pete Buttigieg           112
Andrew Yang        Bernie Sanders             8
                   Beto O'Rourke              3
                   Elizabeth Warren           4
                   John Delaney               1
                   John Hickenlooper          2
                   Kamala Harris              2
                   Pete Buttigieg             4
                   Tulsi Gabbard              2
Bernie Sanders     Beto O'Rourke             22
                   Cory Booker                9
                   Elizabeth Warren         138
                   Jay Inslee                13
                   John Hickenlooper          4
                   Julián Castro              2
                   Kamala Harris             38
                   Kirsten Gillibrand         4
                   Pete Buttigieg            31
                   Tulsi Gabbard             65
...                                         ...
Elizabeth Warren   Kamala Harris            121
                   Kirsten Gillibrand        32
                   Pete Buttigieg            75
                   Tulsi Gabbard              6
                   Wayne Messam               1
Jay Inslee         John Delaney               1
                   John Hickenlooper          7
                   Julián Castro              3
                   Kamala Harris             22
                   Kirsten Gillibrand         8
                   Pete Buttigieg            42
John Delaney       John Hickenlooper          1
                   Kamala Harris              4
                   Kirsten Gillibrand         1
                   Pete Buttigieg             4
John Hickenlooper  Julián Castro              2
                   Kamala Harris             15
                   Kirsten Gillibrand        11
                   Pete Buttigieg            15
Julián Castro      Kamala Harris             20
                   Kirsten Gillibrand         5
                   Pete Buttigieg            11
                   Tulsi Gabbard              2
Kamala Harris      Kirsten Gillibrand       130
                   Pete Buttigieg           170
                   Tulsi Gabbard              6
                   Wayne Messam               1
Kirsten Gillibrand Pete Buttigieg            26
                   Tulsi Gabbard              2
Pete Buttigieg     Tulsi Gabbard              3

[82 rows x 1 columns]

In [32]:
pair_counts = (
    candidate_pairs
    .groupby([
        "candidate_x",
        "candidate_y",
    ])
    .size()
    .to_frame("count")
    .sort_values("count", ascending = False)
    .reset_index()
)

pair_counts.to_csv(
    "../outputs/candidate-pair-counts.csv",
    index = False
)

pair_counts.loc[lambda df: df["count"] >= 50]

,candidate_x,candidate_y,count
0,Kamala Harris,Pete Buttigieg,170
1,Cory Booker,Kamala Harris,166
2,Beto O'Rourke,Pete Buttigieg,144
3,Amy Klobuchar,Kamala Harris,141
4,Bernie Sanders,Elizabeth Warren,138
5,Kamala Harris,Kirsten Gillibrand,130
6,Beto O'Rourke,Kamala Harris,128
7,Elizabeth Warren,Kamala Harris,121
8,Amy Klobuchar,Pete Buttigieg,112
9,Cory Booker,Kirsten Gillibrand,80


## Identify the most common three-candidate combinations

In [35]:
triplet_counts = (
    candidate_triplets
    .groupby([
        "candidate_x",
        "candidate_y",
        "candidate_z",
    ])
    .size()
    .to_frame("count")
    .sort_values("count", ascending = False)
    .reset_index()
)

triplet_counts.to_csv(
    "../outputs/candidate-triplet-counts.csv",
    index = False
)

triplet_counts.loc[lambda df: df["count"] >= 10]

,candidate_x,candidate_y,candidate_z,count
0,Cory Booker,Kamala Harris,Kirsten Gillibrand,33
1,Amy Klobuchar,Kamala Harris,Kirsten Gillibrand,27
2,Beto O'Rourke,Kamala Harris,Pete Buttigieg,26
3,Amy Klobuchar,Kamala Harris,Pete Buttigieg,22
4,Amy Klobuchar,Beto O'Rourke,Pete Buttigieg,19
5,Elizabeth Warren,Kamala Harris,Pete Buttigieg,17
6,Amy Klobuchar,Elizabeth Warren,Kamala Harris,17
7,Amy Klobuchar,Cory Booker,Kamala Harris,17
8,Elizabeth Warren,Kamala Harris,Kirsten Gillibrand,16
9,Cory Booker,Kamala Harris,Pete Buttigieg,14


## Count number of donors who gave to at least three female candidates

In [36]:
(
    contributor_totals
    .loc[lambda df: df["Candidate Name"].isin(
        candidates
        .loc[lambda df: df["Candidate Sex"] == "F"]
        ["Candidate Name"]
    )]
    .groupby([ "donor_id" ])
    ["Candidate Name"]
    .nunique()
    .loc[lambda x: x >= 3]
    .pipe(len)
)

44

## Calculate number of donors, per candidate, that gave to multiple campaigns

In [37]:
# Added by Steve
candidate_pairs.head()

,donor_id,candidate_x,candidate_y
8287,COLLIER|PERRY|76567,Amy Klobuchar,Andrew Yang
18921,JEAN|YNGVE|46304,Amy Klobuchar,Bernie Sanders
23339,KAREN|ALLIN|30305,Amy Klobuchar,Bernie Sanders
24146,KATHY|GIBBONS|20008,Amy Klobuchar,Bernie Sanders
29269,MARK|MOLLOY|50214,Amy Klobuchar,Bernie Sanders


In [34]:
# Added by Steve
candidate_pairs.melt(id_vars = ["donor_id"], value_name = "candidate").

,donor_id,variable,candidate
0,COLLIER|PERRY|76567,candidate_x,Amy Klobuchar
1,JEAN|YNGVE|46304,candidate_x,Amy Klobuchar
2,KAREN|ALLIN|30305,candidate_x,Amy Klobuchar
3,KATHY|GIBBONS|20008,candidate_x,Amy Klobuchar
4,MARK|MOLLOY|50214,candidate_x,Amy Klobuchar
5,MARK|ROTHACHER|84117,candidate_x,Amy Klobuchar
6,MARK|WIZNITZER|22205,candidate_x,Amy Klobuchar
7,PARKE|CAPSHAW|22902,candidate_x,Amy Klobuchar
8,ALICE|JARCHO|10065,candidate_x,Amy Klobuchar
9,AMY|LOFGREN|85250,candidate_x,Amy Klobuchar


In [42]:
# Added by Steve, what does melt do?

print(candidate_pairs.head())
candidate_pairs.melt(id_vars = ["donor_id"], value_name = "candidate").head()

                  donor_id    candidate_x     candidate_y
8287   COLLIER|PERRY|76567  Amy Klobuchar     Andrew Yang
18921     JEAN|YNGVE|46304  Amy Klobuchar  Bernie Sanders
23339    KAREN|ALLIN|30305  Amy Klobuchar  Bernie Sanders
24146  KATHY|GIBBONS|20008  Amy Klobuchar  Bernie Sanders
29269    MARK|MOLLOY|50214  Amy Klobuchar  Bernie Sanders


,donor_id,variable,candidate
0,COLLIER|PERRY|76567,candidate_x,Amy Klobuchar
1,JEAN|YNGVE|46304,candidate_x,Amy Klobuchar
2,KAREN|ALLIN|30305,candidate_x,Amy Klobuchar
3,KATHY|GIBBONS|20008,candidate_x,Amy Klobuchar
4,MARK|MOLLOY|50214,candidate_x,Amy Klobuchar


In [44]:
# Added by Steve, what's going on?
# Distinct Donor IDs are the number of donors who donated to the candidate
(
    candidate_pairs
        .melt(
            id_vars = [ "donor_id" ],
            value_name = "candidate"
        )
        .groupby([ "candidate" ])
        ["donor_id"]
        .nunique()
        .sort_values(ascending = False)
        .to_frame("Multiple-Candidate Donors")
        .join(
            distinct_donor_counts,
            how = "left"
        )
)

,Multiple-Candidate Donors,Distinct Donor IDs
candidate,,
Kamala Harris,722,7489
Pete Buttigieg,512,4045
Elizabeth Warren,420,3177
Beto O'Rourke,394,4879
Amy Klobuchar,386,2867
Cory Booker,313,3063
Bernie Sanders,296,9321
Kirsten Gillibrand,277,1723
Jay Inslee,128,1214


In [38]:
(
    candidate_pairs
    .melt(
        id_vars = [ "donor_id" ],
        value_name = "candidate"
    )
    .groupby([ "candidate" ])
    ["donor_id"]
    .nunique()
    .sort_values(ascending = False)
    .to_frame("Multiple-Candidate Donors")
    .join(
        distinct_donor_counts,
        how = "left"
    )
    .assign(**{
        "Per 1k": lambda df: (
            df["Multiple-Candidate Donors"] * 1000 /
            df["Distinct Donor IDs"]
        ).round(1)
    })
)

,Multiple-Candidate Donors,Distinct Donor IDs,Per 1k
candidate,,,
Kamala Harris,722,7489,96.4
Pete Buttigieg,512,4045,126.6
Elizabeth Warren,420,3177,132.2
Beto O'Rourke,394,4879,80.8
Amy Klobuchar,386,2867,134.6
Cory Booker,313,3063,102.2
Bernie Sanders,296,9321,31.8
Kirsten Gillibrand,277,1723,160.8
Jay Inslee,128,1214,105.4


## Calculate total number of donors observed giving to multiple campaigns

In [29]:
donor_candidate_counts = (
    contributor_totals
    .groupby(["donor_id"])
    .size()
    .to_frame("num_candidates")
    .reset_index()
)

(
    donor_candidate_counts
    ["num_candidates"]
    .value_counts()
    .sort_index()
)

1    37620
2     1401
3      203
4       55
5        8
6        3
Name: num_candidates, dtype: int64

... and as a proportion of the total:

In [21]:
(
    donor_candidate_counts
    ["num_candidates"]
    .value_counts(normalize = True)
    .sort_index()
)

1    0.957496
2    0.035658
3    0.005167
4    0.001400
5    0.000204
6    0.000076
Name: num_candidates, dtype: float64

---

---

---